# 2. Validação dos Dados Gerados

Este notebook executa verificações de consistência na planilha gerada pelo processo anterior.

**Objetivo:** Garantir que não existam erros como nomes não encontrados, mensalidades nulas indevidas ou duplicatas.

**Pré-requisito:** Ter executado com sucesso o notebook `1_Gerar_Nova_Planilha.ipynb`.

In [ ]:
import pandas as pd

### 2.1 Carregamento dos Dados para Validação
Carrega o arquivo gerado (`dados_atualizados.xlsx`) e a planilha original de mensalidades para comparação.

In [ ]:
dados = pd.read_excel('../01_Processamento/dados_atualizados.xlsx', index_col=0)
mensalidades = pd.read_excel('../Data/mensalidades.xlsx', index_col=0)

### 2.2 Verificação de Mensalidades Nulas
Verifica se existem pessoas com nome preenchido mas sem valor de mensalidade.

In [ ]:
# Filter out rows where 'NOME' is NaN
dados_filtered = dados.dropna(subset=['NOME'])

# Count null values in 'MENSALIDADE' column for filtered data
null_mensalidades_count = dados_filtered['MENSALIDADE'].isnull().sum()

print(f"O número de valores nulos na coluna MENSALIDADE (onde NOME não é NaN) é: {null_mensalidades_count}")

In [ ]:
# Filter rows where 'MENSALIDADE' is NaN and 'NOME' is not NaN
nomes_mensalidade_nan = dados[(dados['MENSALIDADE'].isnull()) & (dados['NOME'].notnull())]

# Print the 'NOME' column for the filtered rows
print("Nomes com MENSALIDADE nula:")
for nome in nomes_mensalidade_nan['NOME']:
    print(nome)

### 2.3 Verificação de Nomes Não Encontrados (Mensalidades -> Dados)
Verifica se há nomes na planilha de mensalidades que não foram encontrados na nossa planilha de dados. Isso pode indicar novos beneficiários ou erros de digitação.

In [ ]:
# Get the names from both dataframes
nomes_mensalidades = mensalidades['Nome'].unique()
nomes_dados = dados['NOME'].unique()

# Find names in 'mensalidades' that are not in 'dados'
nomes_nao_encontrados = [nome for nome in nomes_mensalidades if nome not in nomes_dados]

# Print the names not found
if nomes_nao_encontrados:
    print("Names in 'mensalidades' not found in 'dados':")
    for nome in nomes_nao_encontrados:
        print(nome)
else:
    print("All names in 'mensalidades' are present in 'dados'.")

### 2.4 Verificação de Nomes Sobrando (Dados -> Mensalidades)
Verifica nomes que estão na nossa planilha de dados mas não aparecem na planilha de mensalidades (quem não está sendo cobrado).

In [ ]:
# Get the names from both dataframes
nomes_mensalidades = mensalidades['Nome'].unique()
nomes_dados = dados['NOME'].unique()

# Find names in 'dados' that are not in 'mensalidades'
nomes_nao_encontrados_dados = [nome for nome in nomes_dados if nome not in nomes_mensalidades]

# Print the names not found
if nomes_nao_encontrados_dados:
    print("Names in 'dados' not found in 'mensalidades':")
    for nome in nomes_nao_encontrados_dados:
        print(nome)
else:
    print("All names in 'dados' are present in 'mensalidades'.")

### 2.5 Verificação de Duplicatas
Verifica se há nomes duplicados nas planilhas.

In [ ]:
# Check for duplicate names in the 'NOME' column of the 'dados' DataFrame, excluding NaN values
duplicate_names = dados[dados['NOME'].notna() & dados['NOME'].duplicated(keep=False)]

# Display the rows with duplicate names
if not duplicate_names.empty:
    print("Nomes duplicados encontrados na planilha 'dados' (excluindo NaN):")
    display(duplicate_names)
else:
    print("Não há nomes duplicados na planilha 'dados' (excluindo NaN).")

In [ ]:
# Filter 'dados' for rows where 'MENSALIDADE' is not NaN
dados_mensalidade_not_null = dados[dados['MENSALIDADE'].notna()]

# Get the names from the filtered 'dados' DataFrame
nomes_dados_mensalidade_not_null = dados_mensalidade_not_null['NOME'].tolist()

# Get the names from the 'mensalidades' DataFrame
nomes_mensalidades = mensalidades['Nome'].tolist()

# Find names in 'dados' with non-null 'MENSALIDADE' that are not in 'mensalidades'
names_in_dados_not_in_mensalidades = [
    nome for nome in nomes_dados_mensalidade_not_null if nome not in nomes_mensalidades
]

# Print the names found
if names_in_dados_not_in_mensalidades:
    print("Nomes presentes em 'dados' (com MENSALIDADE não nula) e não presentes em 'mensalidades':")
    for nome in names_in_dados_not_in_mensalidades:
        print(nome)
else:
    print("Não há nomes em 'dados' (com MENSALIDADE não nula) que não estejam em 'mensalidades'.")

In [ ]:
# Filter out rows where 'Data Nasc.' is NaN
mensalidades_filtered = mensalidades.dropna(subset=['Data Nasc.'])

# Count the occurrences of each name in the filtered DataFrame
name_counts = mensalidades_filtered['Nome'].value_counts()

# Filter for names that appear more than once
duplicate_names_counts = name_counts[name_counts > 1]

# Display the counts of duplicate names
if not duplicate_names_counts.empty:
    print("Contagem de nomes duplicados encontrados na planilha 'mensalidades' (excluindo Data Nasc. nulo):")
    display(duplicate_names_counts)
else:
    print("Não há nomes duplicados na planilha 'mensalidades' (excluindo Data Nasc. nulo).")

In [ ]:
# Filter out rows where 'Data Nasc.' is NaN
mensalidades_filtered = mensalidades.dropna(subset=['Data Nasc.'])

# Check for duplicate names in the 'Nome' column of the filtered DataFrame
duplicate_names_mensalidades = mensalidades_filtered[mensalidades_filtered['Nome'].duplicated(keep=False)]

# Display the rows with duplicate names
if not duplicate_names_mensalidades.empty:
    print("Nomes duplicados encontrados na planilha 'mensalidades' (excluindo Data Nasc. nulo):")
    display(duplicate_names_mensalidades)
else:
    print("Não há nomes duplicados na planilha 'mensalidades' (excluindo Data Nasc. nulo).")